In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib
import re
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pathlib import Path
import csv
import os
import pprint

In [2]:
def get_data_url(target_month, place):
    year = target_month.strftime("%Y")
    month = target_month.strftime("%m")
    base = 'https://keirin.kdreams.jp'
    baseAdd1 = "/" + place + "/schedule/" + year + "/" + month
    url = base + baseAdd1
    return urllib.parse.quote_plus(url, "/:?=&")

In [3]:
def month_span(start_month, end_month):
    """start_date、end_dateの期間に含まれる日毎のdatetimeオブジェクトを返すジェネレータ
    """
    i = 0
    while 1 :
        target_month = start_month + relativedelta(months=i)
        if target_month > end_month: break
        yield target_month
        i += 1

In [4]:
def parse_racedetail_to_csv(racedetail_links, kaisaidate, place, filename):

    for i, racedetail_link in enumerate(racedetail_links):

        race_num = str(i + 1)
        
        headers = {'User-Agent': 'Mozilla/5.0'}
        time.sleep(sleep_time)
        try:
            response = requests.get(racedetail_link, headers=headers)
        except:
            print("Response error:", racedetail_link)
            continue
            
        soup = BeautifulSoup(response.text, 'html.parser')

        print("RACE RESULT", kaisaidate, place, race_num)

        result_bs = soup.find("table", class_="result_table")
        trs = result_bs.find_all("tr")
        trs.pop(0)
        race_result = []
        for tr in trs:
            result_rows = []
            tds = tr.find_all("td")
            for td in tds:
                result_rows.append(td.text)
            #print(result_rows)
            # make list to store result by order
            race_result.append(result_rows[2])

        print(race_result)

        print("RACE RACORD")
        racecard_bs = soup.find("table", class_="racecard_table")
        #print(racecard)
        riders_bs = racecard_bs.find_all("tr", class_=re.compile("^n"))
        for rider_bs in riders_bs:
            rows = []
            for td in rider_bs.find_all("td"):
                row = td.text
                row = ''.join(row.split())
                rows.append(row)
            if len(rows) < 23:
                rows.insert(3, bracket)

            names = rider_bs.find("td", class_="rider bdr_r").get_text().strip().splitlines()
            name = names[0]
            homes = ''.join(names[1].split()).split("/")
            prefecture = homes[0]
            age = homes[1]
            period = homes[2]

            rows.pop(5)
            rows.insert(5, name)
            rows.insert(6, prefecture)
            rows.insert(7, age)
            rows.insert(8, period)

            #print(rows)

            rows[0] = rows[0].replace('◎', '1').replace('○', '2').replace('△', '3').replace('▲', '4').replace('×', '5').replace('注', '6')
            bracket = rows[3]
            rows[6] = rows[6].replace('北海道', '1').replace('青森', '2').replace('岩手', '3').replace('宮城', '4').replace('秋田', '5').replace('山形', '6').replace('福島', '7').replace('茨城', '8').replace('栃木', '9').replace('群馬', '10').replace('埼玉', '11').replace('千葉', '12').replace('東京', '13').replace('神奈川', '14').replace('新潟', '15').replace('富山', '16').replace('石川', '17').replace('福井', '18').replace('山梨', '19').replace('長野', '20').replace('岐阜', '21').replace('静岡', '22').replace('愛知', '23').replace('三重', '24').replace('滋賀', '25').replace('京都', '26').replace('大阪', '27').replace('兵庫', '28').replace('奈良', '29').replace('和歌山', '30').replace('鳥取', '31').replace('島根', '32').replace('岡山', '33').replace('広島', '34').replace('山口', '35').replace('徳島', '36').replace('香川', '37').replace('愛媛', '38').replace('高知', '39').replace('福岡', '40').replace('佐賀', '41').replace('長崎', '42').replace('熊本', '43').replace('大分', '44').replace('宮崎', '45').replace('鹿児島', '46').replace('沖縄', '47')
            rows[9] = rows[9].replace('S1', '1').replace('S2', '2').replace('A1', '3').replace('A2', '4').replace('A3', '5')
            rows[10] = rows[10].replace('両', '1').replace('逃', '2').replace('追', '3')

            car_num = rows[4]
            result = race_result.index(car_num) + 1
            rows.append(result)

            rows.insert(0, kaisaidate)
            rows.insert(1, place)
            rows.insert(2, race_num)

            print(rows)

            with open(filename, "a") as f:
                    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
                    writer.writerow(rows)     # list（1次元配列）の場合

In [5]:
filename = "test.csv"
if os.path.exists(filename):
    os.remove(filename)
    
csv_header = [''] * 30
csv_header[0] = 'date'
csv_header[1] = 'place'
csv_header[2] = 'race_num'
csv_header[3] = 'predict'
csv_header[4] = 'koukiai'
csv_header[5] = 'evaluation'
csv_header[6] = 'bracket'
csv_header[7] = 'car_num'
csv_header[8] = 'name'
csv_header[9] = 'prefecture'
csv_header[10] = 'age'
csv_header[11] = 'period'
csv_header[12] = 'rank'
csv_header[13] = 'leg'
csv_header[14] = 'gear'
csv_header[15] = 'racing piont'
csv_header[16] = 'S'
csv_header[17] = 'B'
csv_header[18] = 'Nige'
csv_header[19] = 'Maki'
csv_header[20] = 'Sashi'
csv_header[21] = 'Ma'
csv_header[22] = '1st'
csv_header[23] = '2nd'
csv_header[24] = '3rd'
csv_header[25] = 'Chakugai'
csv_header[26] = 'win'
csv_header[27] = '2ren'
csv_header[28] = '3ren'
csv_header[29] = 'result'

with open(filename, "a") as f:
        writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
        writer.writerow(csv_header)   # list（1次元配列）の場合

In [6]:
# wait 5 seconds before access url
sleep_time = 5

start_month = datetime.strptime('201501', '%Y%m')
end_month = datetime.strptime('201808', '%Y%m')

In [7]:
places = ['hiratsuka']

In [ ]:
for target_month in month_span(start_month, end_month):
    for place in places:
        target_url = get_data_url(target_month, place)
        print(target_url)
        headers = {'User-Agent': 'Mozilla/5.0'}
        time.sleep(sleep_time)
        try:
            response = requests.get(target_url, headers=headers)# <Response [200]>
        except:
            print("Response error:", target_url)
            continue
            
        soup = BeautifulSoup(response.text, 'html.parser')
        a_results = soup.find_all("a", string="結果")

        for a_result in a_results:
            raceresult_link = a_result.get("href")
            print(raceresult_link)
            time.sleep(sleep_time)
            try:
                response = requests.get(raceresult_link, headers=headers)
            except:
                print("Response error:", raceresult_link)
                continue
                
            soup = BeautifulSoup(response.text, 'html.parser')

            lis = soup.find("ul", id="JS_UL_KAISAI_DATE_TAB").find_all("li")
            kaisaidates = []
            for li in lis:
                kaisaidates.append(li.get("kaisaidate"))

            for kaisaidate in kaisaidates:
                # get race detail base URL
                racedetail_id = "JS_DL_KAISAI_DETAIL_INFO_NAV_" + kaisaidate
                racedetail_link_base = soup.find("dl", id=racedetail_id).find("a", string="レース詳細").get("href")
                print(racedetail_link_base)

                # get racedetail URLs of all races
                time.sleep(sleep_time)
                try:
                    response = requests.get(racedetail_link_base, headers=headers)
                except:
                    print("Response error:", racedetail_link_base)
                    continue
                    
                soup = BeautifulSoup(response.text, 'html.parser')

                dl_id = "JS_DL_KAISAI_DETAIL_RACE_NAV_" + kaisaidate
                a_races = soup.find("dl", id=dl_id).find_all("a")

                racedetail_links = []
                for a_race in a_races:
                    href = a_race.get("href")
                    racedetail_links.append(href)
                racedetail_links[0] = racedetail_link_base
                
                # call function - parse racedetail URL and save racer data and result to csv file
                parse_racedetail_to_csv(racedetail_links, kaisaidate, place, filename)
                

https://keirin.kdreams.jp/hiratsuka/schedule/2015/01
https://keirin.kdreams.jp/hiratsuka/raceresult/35201501100100/
https://keirin.kdreams.jp/hiratsuka/racedetail/3520150110010001/?l-id=l-pc-srri-srdi-raceinfo_kaisai_detail_info_nav_btn
RACE RESULT 20150110 hiratsuka 1
['1', '2', '3', '7', '6', '4', '5', '9', '8']
RACE RACORD
['20150110', 'hiratsuka', '1', '', '', '', '1', '1', '山口 茂', '7', '35', '98', '3', '2', '3.92', '90.00', '4', '3', '2', '10', '0', '0', '5', '7', '3', '7', '22.7', '54.5', '68.1', 1]
['20150110', 'hiratsuka', '1', '', '', '', '2', '2', '黒田 篤', '9', '35', '88', '3', '1', '3.92', '87.54', '3', '0', '0', '0', '5', '1', '2', '4', '4', '14', '8.3', '25.0', '41.6', 2]
['20150110', 'hiratsuka', '1', '', '', '', '3', '3', '木田 有隆', '7', '29', '92', '3', '3', '3.92', '85.13', '2', '0', '0', '0', '2', '0', '2', '0', '2', '18', '9.0', '9.0', '18.1', 3]
['20150110', 'hiratsuka', '1', '', '', '', '4', '4', '伊藤 太一', '19', '32', '91', '4', '1', '3.92', '81.70', '4', '4', '0', '2'

RACE RESULT 20150110 hiratsuka 6
['5', '8', '6', '4', '1', '7', '2', '3', '9']
RACE RACORD
['20150110', 'hiratsuka', '6', '', '', '', '1', '1', '齋藤 明', '1', '47', '61', '3', '3', '3.92', '95.36', '0', '0', '0', '0', '1', '0', '1', '0', '2', '23', '3.8', '3.8', '11.5', 5]
['20150110', 'hiratsuka', '6', '', '', '', '2', '2', '長崎 達也', '14', '33', '89', '3', '1', '3.92', '96.73', '0', '0', '0', '0', '0', '1', '0', '1', '1', '13', '0.0', '6.6', '13.3', 7]
['20150110', 'hiratsuka', '6', '', '', '', '3', '3', '笹川 竜治', '15', '42', '71', '3', '3', '3.86', '97.60', '0', '0', '0', '0', '0', '1', '0', '1', '3', '22', '0.0', '3.8', '15.3', 8]
['20150110', 'hiratsuka', '6', '', '', '', '4', '4', '小林 圭介', '8', '38', '83', '3', '3', '3.92', '93.05', '0', '0', '0', '0', '5', '1', '3', '3', '5', '7', '16.6', '33.3', '61.1', 4]
['20150110', 'hiratsuka', '6', '', '', '', '4', '5', '北野 良栄', '12', '31', '95', '3', '2', '3.92', '100.60', '2', '20', '8', '2', '0', '0', '6', '4', '6', '19', '17.1', '28.5', '45

In [ ]:
pd.read_csv("test.csv", encoding="SHIFT_JIS", header=None)